In [1]:
print_imports()

Loaded modules:
numpy                np              1.14.0
pandas               pd              0.22.0
sklearn              sk              0.19.1
keras                ks              2.1.4

matplotlib           mpl             2.1.2
matplotlib.pyplot    plt             N/A
matplotlib.image     mpimg           N/A
seaborn              sns             0.8.1
PIL                  PIL             5.0.0

ExergyUtilities      exergy          2.0.

pyspark              pyspark         2.2.1


### Imports

In [36]:
import os, re, logging, json
from pprint import pprint
from keras import backend as K
from datetime import datetime

In [3]:
mod_path = r"/home/batman/git/ai_drive/src"
sys.path.append(mod_path)
logging.debug("ADDED TO PATH: ".format(mod_path))

In [4]:
import drive.analysis_offline as analysis

### Paths

In [5]:
this_project_path = r"/media/batman/USB STICK"
project_name = r'catdog1'

### Plotting function

In [6]:
model_title = "10 Epochs"
def plot_hist(history_dict):
    #fig = plt.figure(figsize=(5,4))
    #fig=plt.figure(figsize=(20, 10),facecolor='white')

    #f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 5),sharey=False,facecolor='white')
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 5),sharey=False,facecolor='0.15')
    
    ax1.plot(history_dict['epoch'],  history_dict['history']['loss'],label="Train")
    ax1.plot(history_dict['epoch'],  history_dict['history']['val_loss'],label="CV")
    ax1.set_title("Loss function development - Training set vs CV set")
    ax1.legend(loc='upper right')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Values')
    
    ax2.plot(history_dict['epoch'],  history_dict['history']['acc'],label="Train")
    ax2.plot(history_dict['epoch'],  history_dict['history']['val_acc'],label="CV")
    ax2.set_title("Accuracy development - Training set vs CV set")
    ax2.legend(loc='upper right')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Values')
    
    plt.suptitle(model_title, fontsize=16)
    
    plt.show()

#plot_hist(history_dict)

```total_params = (filter_height * filter_width * input_image_channels + 1) * number_of_filters```

### Each loaded layer has a summary string

In [7]:
def Conv2D(params):
    #print(params['class_name'])
    #pprint(params['config'])
    kernel_dim = (params['config']['kernel_size'][0])
    filters = (params['config']['filters'])
    return "{}, kernel {}, filters {}".format(params['class_name'],kernel_dim,filters)

def MaxPooling2D(params):
    #print(params['class_name'])    
    #pprint(params['config'])
    pool_size = (params['config']['pool_size'][0])
    
    return "{}, pool {}".format(params['class_name'],pool_size)
def Flatten(params):
    #print(params['class_name'])
    return "{}".format(params['class_name'])

def Dropout(params):
    #print(params['class_name'])
    #pprint(params['config'])
    drp_rate = (params['config']['rate'])
    return "{}, dropout {}".format(params['class_name'],drp_rate)
    
    
    #raise
def Dense(params):
    #print(params['class_name'])
    return "{}".format(params['class_name'])

layer_funcs = {
        'Conv2D':Conv2D,
        'MaxPooling2D':MaxPooling2D,
        'Flatten':Flatten,
        'Dropout':Dropout,
        'Dense':Dense,
}

### Parameter counting

In [8]:
def count_params(model):
    """Uses backend 'K' count_params
    Returns dict for Total/Trainable/Non-trainable
    """
    trainable_count = int(
        np.sum([K.count_params(p) for p in set(model.trainable_weights)]))
    non_trainable_count = int(
        np.sum([K.count_params(p) for p in set(model.non_trainable_weights)]))
    return {'Total':trainable_count + non_trainable_count,'Trainable':trainable_count,'Non-trainable':non_trainable_count}
    #print('Total params: {:,}'.format(trainable_count + non_trainable_count))
    #print('Trainable params: {:,}'.format(trainable_count))
    #print('Non-trainable params: {:,}'.format(non_trainable_count))

In [31]:
%%script false
import re
date_re = re.compile('(?P<a_year>\d{2,4})-(?P<a_month>\d{2})-(?P<a_day>\d{2}) (?P<an_hour>\d{2}):(?P<a_minute>\d{2}):(?P<a_second>\d{2}[.\d]*)')
found = date_re.match('2016-02-29 12:34:56.789')
if found is not None:
    print found.groupdict()
#{'a_year': '2016', 'a_second': '56.789', 'a_day': '29', 'a_minute': '34', 'an_hour': '12', 'a_month': '02'}
found.groupdict()['a_month']
#'02'

In [42]:
def create_date(res_dict):
    #values = ['2014', '08', '17', '18', '01', '05']

    datevec = [ int(res_dict['year']),
                int(res_dict['month']),
                int(res_dict['day']),
                int(res_dict['hour']),
                int(res_dict['minute']),
                int(res_dict['second']),
              ]
    
    this_dt = datetime(*datevec)
    return this_dt


In [75]:
def parse_log_string(l):
    """datetime, log string text, module string
    """
    #log_regex = re.compile(r"(?P<year>\d{2,4})-(?P<month>\d{2})-(?P<day>\d{2}) (?P<hour>\d{2}):(?P<minute>\d{2}):(?P<second>\d{2}[.\d]*).*:\s(?P<logstring>.*)")
    #-- TODO !!!!
    log_regex = re.compile(r"(?P<year>\d{2,4})-(?P<month>\d{2})-(?P<day>\d{2}) (?P<hour>\d{2}):(?P<minute>\d{2}):(?P<second>\d{2}),[\d]*\s-\s(?P<level>\d\d)\s*-\s(?P<module_str>.*).*:\s(?P<logstring>.*)$")
    #log_regex = re.compile(r"(?P<year>\d{2,4})-(?P<month>\d{2})-(?P<day>\d{2}) (?P<hour>\d{2}):(?P<minute>\d{2}):(?P<second>\d{2}),[\d]*\s-\s(?P<level>\d\d)\s*-\s*(?P<modules>.*)\s:\s(?P<logstring>.*)$")
    res = log_regex.match(l)
    res_dict = res.groupdict()
    
    logstr = res_dict.pop('logstring', None)
    module_str = res_dict.pop('module_str', None)
        #print(res_dict)
    this_dt = create_date(res_dict)    
    
    return({'dt':this_dt,'logstr':logstr,'modstr':module_str})

In [102]:
def get_log_file(this_run_path):
    log_file = [os.path.join(this_run_path,f) for f in os.listdir(this_run_path) 
             if re.match('^log.txt$',f)][0]
    assert os.path.exists(log_file)
    
    with open(log_file) as f:
        #for l in lines:
            #print(l)
        lines = f.readlines()
        #content = [x.strip() for x in content] 
    
    start_time = parse_log_string(lines[0])['dt']
    end_time = parse_log_string(lines[-1])['dt']
    elapsed = end_time-start_time
    finish_found = None
    generator = None
    #for l in lines[-100:-1]:
    for l in lines[0:100]:        
        l = l.strip()
        res_dict = parse_log_string(l)
        
        # Get the image generator
        if not generator: 
            gen_regex = re.compile(r"get_train_generator")
            generator_line = gen_regex.search(res_dict['modstr'])
            #generator_line = gen_regex.match(res_dict['logstr'])
            if generator_line:
                #print(generator_line.group())
                #print()
                generator = res_dict['modstr']
                #raise
            #my_generators         
    
        
        
        # Check if finished
        finished_regex = re.compile(r"Finished training")
        finished_line = finished_regex.match(res_dict['logstr'])
        if finished_line: 
            finish_found=True

    
    return({'finished':finish_found,'start':start_time,'elapsed':finish_found,'generator':generator})
    #assert(finish_found)


### Main

In [103]:
root_path = os.path.join(this_project_path,project_name)
run_folders = [dir for dir in os.listdir(root_path) if re.match('run',dir)]
run_folders.sort()

# Loop run folders
for rfolder in run_folders:
    this_run_path = os.path.join(root_path,rfolder)
    logging.debug('**** RUN {} ****'.format(rfolder))
    
    ###### Log file ######
    log = get_log_file(this_run_path)
    
    #raise
    ###### Weights ######
    wts = analysis.get_weights(this_run_path)
    if wts: best_wt = wts[-1] # BEST weight (last weight)

    ###### Architecture ######
    model = analysis.load_model(this_run_path)
    arch_dict = analysis.read_model_json(this_run_path)
    
    #model.summary()
    print(count_params(model))
    
    ##### Reload weights #####
    if wts:
        print(best_wt)
        model.load_weights(best_wt['path'])
    #raise
    ###### Loop layers ######
    for i,layer in enumerate(arch_dict['config']):
        layer_str = layer_funcs[layer['class_name']](layer)
        print(i,layer_str)

    ###### History ######
    path_hist = analysis.get_history(this_run_path)    
    with open(path_hist) as hist_file:
        hist_dict = json.load(hist_file)
    print("Epochs",hist_dict['params']['epochs'])
    print("Steps",hist_dict['params']['steps'])
    
    #plot_hist(hist_dict)
    del model
    raise


root  - 10  - <ipython-input-103-a341910f95ab>  <module>                      : **** RUN run078 ****


RuntimeError: No active exception to reraise